# Credit card fraud detector

In this solution we will build the core of a credit card fraud detection system using SageMaker. We will start by training an anomaly detection algorithm, then proceed to train different NN models for supervised training. To deal with the highly unbalanced data common in fraud detection, our first model will use re-weighting of the data, and the second will use re-sampling, using the popular SMOTE technique for oversampling the rare fraud data.


## Installs and Dependensies

In [14]:
import sys

sys.path.insert(0, './src/credit-fraud-detection-aws')

In [40]:
import boto3
from zipfile import ZipFile
import yaml
import os

region_name = boto3.Session().region_name
print(f"Region : {region_name} ")

s3 = boto3.resource('s3', region_name=region_name)

# Define the path to your YAML file
file_path = 'config.yml'

try:
    with open(file_path, 'r') as file:
        params = yaml.safe_load(file)

except FileNotFoundError:
    print(f"Error: The file {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Region : us-east-1 


In [24]:
bucket_name = params['bucket_name']
object_key = params['object_key']

# Create S3 object
obj = s3.Object(bucket_name, object_key)

# Download the file to local path
if 'dataset' not in os.listdir():
    os.mkdir('credit-fraud-detection-aws/dataset')

if object_key not in os.listdir('dataset'):
    obj.download_file('dataset/creditcard.csv.zip')
    with ZipFile('dataset/creditcard.csv.zip', 'r') as zf:
        zf.extractall('dataset')

In [26]:
import numpy as np 
import pandas as pd

data = pd.read_csv('dataset/creditcard.csv', delimiter=',')

In [27]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


## Processing Jobs

This script is part of an AWS SageMaker processing step. It prepares credit card transaction data for fraud detection by:

Reading the dataset.

Splitting it into features and labels.

Performing a train/test split.

Saving the processed data for downstream training.

In [28]:
s3.meta.client.upload_file('dataset/creditcard.csv', bucket_name, 'fraud_detection_ann/input/creditcard.csv')


In [29]:
if 'processing' not in os.listdir():
    os.mkdir('processing')
if 'training' not in os.listdir():
    os.mkdir('training')

In [30]:
%%writefile processing/processing.py

import pandas as pd
import numpy as np
import boto3
from sklearn.model_selection import train_test_split

print("Reading data from loacation........")
data = pd.read_csv('/opt/ml/processing/input/creditcard.csv', delimiter=',')

feature_columns = data.columns[:-1]
label_column = data.columns[-1]

features = data[feature_columns].values.astype('float32')
labels = (data[label_column].values).astype('float32')

print(f"Features columns : {feature_columns}")
print(f"Labels columns : {label_column}")

print("Performing train-test split .............")

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.1, random_state=42)

print(f"Training length : {X_train.shape}")
print(f"Test length : {X_test.shape}")

print("Saving Transformed train data to: '/opt/ml/processing/output/train' ...")

np.save("/opt/ml/processing/output/X_train.npy", X_train)
np.save("/opt/ml/processing/output/y_train.npy", y_train)


print("Saving Transformed test data to: '/opt/ml/processing/output/test' ...")

np.save("/opt/ml/processing/output/X_test.npy", X_test)
np.save("/opt/ml/processing/output/y_test.npy", y_test)

print("completed processing ............")


Overwriting processing/processing.py


In [31]:
import boto3

iam = boto3.client('iam')
response = iam.list_roles()
 
for role in response['Roles']:
    if params['role'] in role['RoleName']:
        role_name = role['RoleName']

In [34]:
from sagemaker.processing  import ScriptProcessor,ProcessingInput, ProcessingOutput
import sagemaker

import mlflow

processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve('sklearn', 'us-east-1', '1.2-1'),
    role=role_name,
    command=['python3'],
    instance_type='ml.t3.medium',
    instance_count=1,
    volume_size_in_gb=5
)

processor.run(
    code='processing/processing.py',
    inputs=[ProcessingInput(
        source=f"s3://{bucket_name}/fraud_detection_ann/input/creditcard.csv",
        destination='/opt/ml/processing/input'
    )],
    outputs=[ProcessingOutput(
        source='/opt/ml/processing/output',
        destination=f"s3://{bucket_name}/fraud_detection_ann/output/"
    )]
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2025-11-20-18-53-33-555


..............Reading data from loacation........
Features columns : Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')
Labels columns : Class
Performing train-test split .............
Training length : (256326, 30)
Test length : (28481, 30)
Saving Transformed train data to: '/opt/ml/processing/output/train' ...
Saving Transformed test data to: '/opt/ml/processing/output/test' ...
completed processing ............



## Training

This script (train.py) is the training step in your AWS SageMaker pipeline for credit card fraud detection. It loads the preprocessed data, builds a neural network model, trains it, and integrates with MLflow for experiment tracking.

In [41]:
%%writefile training/train.py

from __future__ import print_function

import argparse
import joblib
import os
import pandas as pd
import keras
import numpy as np

import mlflow


def get_data(train_dir):
    trainx , trainy = np.load(f"{train_dir}/X_train.npy"),np.load(f"{train_dir}/y_train.npy")
    return trainx, trainy

def get_model(trainx):
    
    model = keras.Sequential(
    [
        keras.Input(shape=trainx.shape[1:]),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ])

    return model


def train_model(model,x,y,epochs):
    
    metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),]

    model.compile(
        optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
    )

    callbacks = []

    model.fit(
        x,
        y,
        batch_size=2048,
        epochs=epochs,
        verbose=2,
        callbacks=callbacks,
        validation_split=0.1,
    )

    return model
    

if __name__ == '__main__':
    
    print("Started process.............")
    parser = argparse.ArgumentParser()
    
    print(os.listdir("/opt/ml/input/data/train"))
    
    train_dir = os.environ.get("SM_CHANNEL_TRAIN", "/opt/ml/input/data/train")
    parser.add_argument('--model_dir', type=str, default='/opt/ml/model')
    parser.add_argument('--epochs', type=str, default=10)
    
    args = parser.parse_args()
    x, y = get_data(train_dir)
    
    print(f"x train shape : {x.shape}")
    print(f"y train shape : {y.shape}")
    
    
    # # Set the tracking server URI using the ARN of the tracking server you created
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_ARN'])
    
    # # Enable autologging in MLflow
    mlflow.autolog()

    print("Building model..........")
    model = get_model(x)

    print(model.summary())

    print("Training model..........")

    trained_model = train_model(model,x,y,int(args.epochs))

    print("Completed training........")
    

    

Overwriting training/train.py


In [42]:
%%writefile training/requirements.txt
mlflow-skinny==2.13.2
sagemaker-mlflow==0.1.0

Overwriting training/requirements.txt


In [43]:
from sagemaker.inputs import TrainingInput

prefix = f"s3://{bucket_name}/fraud_detection_ann/output/"
train_x = TrainingInput(prefix, content_type="npy",s3_data_type="S3Prefix")

In [66]:
from sagemaker.tensorflow import TensorFlow

tffdv1 = "tensorflow-fraud-detect-v2"
estimator = TensorFlow(
    entry_point="train.py",
    source_dir='training', # mlflow-integration
    role=role_name,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="2.18",
    py_version="py310",
    use_spot_instances=True,
    base_job_name=tffdv1,
    max_run=3600,
    max_wait=3600*2,
    volume_size=10,
    environment={
        'MLFLOW_TRACKING_ARN': params['mlflow_uri']
    },
    hyperparameters={
        "epochs": 10
    }
)
estimator.fit({"train": train_x})


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-fraud-detect-v2-2025-11-20-20-04-03-554


2025-11-20 20:04:04 Starting - Starting the training job...
2025-11-20 20:04:32 Starting - Preparing the instances for training...
2025-11-20 20:04:56 Downloading - Downloading input data......
2025-11-20 20:05:52 Downloading - Downloading the training image......
2025-11-20 20:06:58 Training - Training image download completed. Training in progress..2025-11-20 20:07:02.833621: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 20:07:07,123 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2025-11-20 20:07:07,125 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-11-20 20:07:07,126 sagemaker-training-toolkit INFO     No Neurons detecte

In [ ]:
loaded_model = mlflow.tensorflow.load_model(model_path)

In [62]:
!pip install mlflow

In [54]:
import mlflow
import keras

In [46]:
mlflow.set_tracking_uri(params['mlflow_uri'])

In [48]:
experiment = mlflow.get_experiment_by_name("Default")

In [49]:
experiment = mlflow.get_run("3909f27fe93c4d89ab86691704f98618")

In [92]:
model = mlflow.keras.load_model("s3://sagemaker-automated-execution-274594569284-us-east-1/0/ad1259068b604ce5902c30cdb74ef999/artifacts/model")

In [82]:
!aws s3 cp s3://sandboxstav/fraud_detection_ann/output/y_test.npy y_test.npy

download: s3://sandboxstav/fraud_detection_ann/output/y_test.npy to ./y_test.npy


In [87]:
import numpy as np
import pandas as pd

In [84]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

In [95]:
eval_data = pd.DataFrame(X_test.copy())
eval_data["label"] = pd.DataFrame(y_test)
f
# Comprehensive evaluation

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [97]:
signature

inputs: 
  [Tensor('float32', (-1, 30))]
outputs: 
  [Tensor('float32', (-1, 1))]
params: 
  None

In [107]:
op = model.predict(X_test)

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [119]:
def predict_wrap(x):
    to_binary = np.vectorize(lambda x: 1 if x >= 0.5 else 0)
    return to_binary(np.squeeze(model.predict(x)))

In [131]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 130.9 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]
      Successfully uninstalled numpy-1.26.4━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [shap]2/3 [shap]]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
amazon-sagemaker-jupyter-ai-q-developer 1.2.7 requires numpy<=2.0.1, but you have numpy 2.2.6 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but

In [ ]:
import shap
result = mlflow.models.evaluate(
    predict_wrap,
    X_test,
    targets=y_test,
    model_type="classifier",
    evaluators=None,
    evaluator_config={"log_explainer": True},
)


2025/11/20 21:07:30 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set the `model_type` and `dataset` arguments correctly.


891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2025/11/20 21:07:32 INFO mlflow.models.evaluation.evaluators.classifier: The evaluation dataset is inferred as binary dataset, positive label is 1.0, negative label is 0.0.
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2025/11/20 21:07:32 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2025/11/20 21:07:35 INFO mlflow.models.evaluation.evaluators.shap: Shap explainer PermutationExplainer is used.


...

191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


PermutationExplainer explainer:   3%|▎         | 59/2000 [04:30<2:43:14,  5.05s/it]

191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


PermutationExplainer explainer:   3%|▎         | 60/2000 [04:35<2:39:46,  4.94s/it]

191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
167/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [123]:
print(f"Accuracy: {result.metrics['accuracy_score']:.3f}")
print(f"F1 Score: {result.metrics['f1_score']:.3f}")

Accuracy: 0.998
F1 Score: 0.000


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 print(f"Accuracy: {result.metrics['accuracy_score']:.3f}")                                   │
│   2 print(f"F1 Score: {result.metrics['f1_score']:.3f}")                                         │
│ ❱ 3 print(f"ROC AUC: {result.metrics['precision']:.3f}")                                         │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'precision'

In [ ]:
def weighted_accuracy(predictions, targets, metrics, sample_weights=None):
    """Custom weighted accuracy metric."""
    if sample_weights is None:
        return (predictions == targets).mean()
    else:
        correct = predictions == targets
        return np.average(correct, weights=sample_weights)


# Create custom metric
custom_accuracy = make_metric(
    eval_fn=weighted_accuracy, greater_is_better=True, name="weighted_accuracy"
)
result = mlflow.models.evaluate(
    model_uri,
    eval_data,
    targets="label",
    model_type="classifier",
    extra_metrics=[custom_accuracy],
)